In [1]:
import torch

from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
from sklearn.utils import shuffle
import math
import sklearn

from sklearn.model_selection import train_test_split

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))



Using cuda device


In [3]:
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F


In [4]:
all_results = {}
all_point_of_fail = {}

In [5]:
bg_data = np.load('training_data/Xj_bg_actual.npy')
sig_data = np.load('training_data/Xj_sig_actual.npy')

In [6]:
import importlib

In [7]:
import my_funcs as mf

importlib.reload(mf)

<module 'my_funcs' from '/home/jerry/particle_nn_analysis/my_funcs.py'>

In [8]:
physics_dataset = mf.Jets_dataset(bg_data, sig_data)

bg:(83475, 100), sig:(98390, 100)


In [9]:
class simple_dataset(Dataset):

    def __init__(self,X, y):
        
        assert len(X) == len(y)
        self.X = torch.Tensor(X)
        self.y = torch.Tensor(y)
                
        
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        sample = {'X': self.X[idx], 'y': self.y[idx]}          
        
        return sample


In [10]:
#[100, 50]

class MyNet_M(torch.nn.Module):
    def __init__(self, input_size):
        
        super(MyNet_M, self).__init__()
        self.input_size = input_size
        self.fc1 = torch.nn.Linear(self.input_size, 100)
        self.fc2 = torch.nn.Linear(100, 50)
        self.fc3 = torch.nn.Linear(50, 2)
        
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        return x


def weights_init(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.xavier_uniform(m.weight.data)

#model.apply(weight_init)

class MyNet_XS(torch.nn.Module):
    def __init__(self, input_size):
        
        super(MyNet_XS, self).__init__()
        self.input_size = input_size
        self.fc1 = torch.nn.Linear(self.input_size, 25)
        self.fc2 = torch.nn.Linear(25, 10)
        self.fc3 = torch.nn.Linear(10, 2)
        
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        return x


class MyNet_S(torch.nn.Module):
    def __init__(self, input_size):
        
        super(MyNet_S, self).__init__()
        self.input_size = input_size
        self.fc1 = torch.nn.Linear(self.input_size, 50)
        self.fc2 = torch.nn.Linear(50, 25)
        self.fc3 = torch.nn.Linear(25, 2)
        
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        return x
    
class MyNet_L(torch.nn.Module):
    def __init__(self, input_size):
        
        super(MyNet_L, self).__init__()
        self.input_size = input_size
        self.fc1 = torch.nn.Linear(self.input_size, 100)
        self.fc2 = torch.nn.Linear(100, 100)
        self.fc3 = torch.nn.Linear(100, 50)
        self.fc4 = torch.nn.Linear(50, 2)
        
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.softmax(self.fc4(x), dim=1)
        return x
    
    
class MyNet_XL(torch.nn.Module):
    def __init__(self, input_size):
        
        super(MyNet_XL, self).__init__()
        self.input_size = input_size
        self.fc1 = torch.nn.Linear(self.input_size, 200)
        self.fc2 = torch.nn.Linear(200, 200)
        self.fc3 = torch.nn.Linear(200, 100)
        self.fc4 = torch.nn.Linear(100, 2)
        
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.softmax(self.fc4(x), dim=1)
        return x

In [11]:
MAX_EPOCHS = 20
loss_fn = torch.nn.CrossEntropyLoss()


In [12]:
def train_epoch(model, dataloader):
    
    model.train()
    
    for batch in dataloader:

        x, y = batch['X'], batch['y']
        x, y = x.to(device), y.to(device)

        # do the forward/backward pass
        model.zero_grad()
        output = model(x)
        loss = loss_fn(output,y)
        loss.backward()
        optimizer.step()

        del x
        del y
    
def validate_acc(model, Xval, Yval):
    
    X = torch.tensor(Xval).float().to(device)
    y_val = torch.tensor(Yval).float().to(device)
    Y_pred = model(X)
    
    Y_pred_rnd = (Y_pred >= 0.5)
    
    matches = sum(Y_pred_rnd == y_val)
    
    acc = matches[0]/(len(Y_pred))

    return acc
    



In [13]:
import pickle

In [14]:

# for sig_ratio in [0.05]:
    
#     X_train,y_train,X_val,y_val = physics_dataset.generate_dataset(sig_noise_ratio=sig_ratio)
    
#     print(f"sig ratio is {sig_ratio}")
    
#     for dimension in [50,100]:
#         print(f"dim is {dimension}")
        
#         this_dim = {
#             'sig_ratio': sig_ratio,
#             'dimension': dimension,
#         }
        
#         this_results = []
        
#         passed = False
        
#         Xtrain, Xval = X_train[:,:dimension], X_val[:,:dimension]
#         Ytrain, Yval = y_train,y_val
        
#         train_dataset = simple_dataset(Xtrain, Ytrain)
        
        
        
#         for trial in tqdm(range(7)):
            
#             train_loader = DataLoader(train_dataset, batch_size=20)
            
#             model = MyNet_M(dimension)
#             model.to(device)

#             optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
            
#             train_acc = 0
#             val_acc = 0
#             success = 0
#             for epoch in (range(MAX_EPOCHS)):
                
#                 train_epoch(model, train_loader)
                
#                 val_acc = validate_acc(model, Xval, Yval)
#                 print(val_acc)
                                
#                 if (val_acc >= 0.6):
#                     success = 1
#                     this_results.append([dimension,epoch])
#                     break
#             if success:
#                 passed = True
                
#             else:
#                 this_results.append([dimension,-1])
                
#         print(this_results)
#         # this_dim['result'] = this_results
        
#         # with open(f'data/sig_{sig_ratio}_dim_{dimension}.pickle', 'wb') as handle:
#         #     pickle.dump(this_dim, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
#         # if passed == False:
#         #     all_point_of_fail[sig_ratio] = dimension
#         #     break
            
#         del model
            



In [15]:
# signalz = [0.001, 0.005, 0.01, 0.02, 0.05, 0.1, 0.15, 0.2, 0.3, 0.5]
# dim_lst = [1, 2, 5, 10, 15, 20, 25, 30, 40, 50, 60, 70, 80, 90, 100]

In [17]:
# for sig_ratio in signalz:
    
#     X_train,y_train,X_val,y_val = physics_dataset.generate_dataset(sig_noise_ratio=sig_ratio)

    
#     print(f"sig ratio is {sig_ratio}")
    
#     for dimension in dim_lst:
#         print(f"dim is {dimension}")
        
#         this_dim = {
#             'sig_ratio': sig_ratio,
#             'dimension': dimension,
#         }
        
#         this_results = []
        
#         passed = False
        
#         Xtrain, Xval = X_train[:,:dimension], X_val[:,:dimension]
#         Ytrain, Yval = y_train,y_val
        
#         train_dataset = simple_dataset(Xtrain, Ytrain)
        
        
        
#         for trial in tqdm(range(4)):
            
#             train_loader = DataLoader(train_dataset, batch_size=20)
            
#             model = MyNet_S(dimension)
#             model.to(device)

#             optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
            
#             train_acc = 0
#             val_acc = 0
#             success = 0
#             for epoch in (range(MAX_EPOCHS)):
                
#                 train_epoch(model, train_loader)
                
#                 val_acc = validate_acc(model, Xval, Yval)
                                
#                 if (val_acc >= 0.6):
#                     success = 1
#                     this_results.append([dimension,epoch])
#                     break
#             if success:
#                 passed = True
                
#             else:
#                 this_results.append([dimension,-1])
                
#         print(this_results)
#         this_dim['result'] = this_results
        
#         with open(f'data3/model_S_sig_{sig_ratio}_dim_{dimension}.pickle', 'wb') as handle:
#             pickle.dump(this_dim, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
#         if passed == False:
#             break
            
#         del model


# for sig_ratio in signalz:
    
#     X_train,y_train,X_val,y_val = physics_dataset.generate_dataset(sig_noise_ratio=sig_ratio)

    
#     print(f"sig ratio is {sig_ratio}")
    
#     for dimension in dim_lst:
#         print(f"dim is {dimension}")
        
#         this_dim = {
#             'sig_ratio': sig_ratio,
#             'dimension': dimension,
#         }
        
#         this_results = []
        
#         passed = False
        
#         Xtrain, Xval = X_train[:,:dimension], X_val[:,:dimension]
#         Ytrain, Yval = y_train,y_val
        
#         train_dataset = simple_dataset(Xtrain, Ytrain)
        
        
        
#         for trial in tqdm(range(4)):
            
#             train_loader = DataLoader(train_dataset, batch_size=20)
            
#             model = MyNet_M(dimension)
#             model.to(device)

#             optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
            
#             train_acc = 0
#             val_acc = 0
#             success = 0
#             for epoch in (range(MAX_EPOCHS)):
                
#                 train_epoch(model, train_loader)
                
#                 val_acc = validate_acc(model, Xval, Yval)
                                
#                 if (val_acc >= 0.6):
#                     success = 1
#                     this_results.append([dimension,epoch])
#                     break
#             if success:
#                 passed = True
                
#             else:
#                 this_results.append([dimension,-1])
                
#         print(this_results)
#         this_dim['result'] = this_results
        
#         with open(f'data3/model_M_sig_{sig_ratio}_dim_{dimension}.pickle', 'wb') as handle:
#             pickle.dump(this_dim, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
#         if passed == False:
#             break
            
#         del model

# for sig_ratio in signalz:
    
#     X_train,y_train,X_val,y_val = physics_dataset.generate_dataset(sig_noise_ratio=sig_ratio)

    
#     print(f"sig ratio is {sig_ratio}")
    
#     for dimension in dim_lst:
#         print(f"dim is {dimension}")
        
#         this_dim = {
#             'sig_ratio': sig_ratio,
#             'dimension': dimension,
#         }
        
#         this_results = []
        
#         passed = False
        
#         Xtrain, Xval = X_train[:,:dimension], X_val[:,:dimension]
#         Ytrain, Yval = y_train,y_val
        
#         train_dataset = simple_dataset(Xtrain, Ytrain)
        
        
        
#         for trial in tqdm(range(4)):
            
#             train_loader = DataLoader(train_dataset, batch_size=20)
            
#             model = MyNet_L(dimension)
#             model.to(device)

#             optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
            
#             train_acc = 0
#             val_acc = 0
#             success = 0
#             for epoch in (range(MAX_EPOCHS)):
                
#                 train_epoch(model, train_loader)
                
#                 val_acc = validate_acc(model, Xval, Yval)
                                
#                 if (val_acc >= 0.6):
#                     success = 1
#                     this_results.append([dimension,epoch])
#                     break
#             if success:
#                 passed = True
                
#             else:
#                 this_results.append([dimension,-1])
                
#         print(this_results)
#         this_dim['result'] = this_results
        
#         with open(f'data3/model_L_sig_{sig_ratio}_dim_{dimension}.pickle', 'wb') as handle:
#             pickle.dump(this_dim, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
#         if passed == False:
#             break
            
#         del model

# for sig_ratio in signalz:
    
#     X_train,y_train,X_val,y_val = physics_dataset.generate_dataset(sig_noise_ratio=sig_ratio)

    
#     print(f"sig ratio is {sig_ratio}")
    
#     for dimension in dim_lst:
#         print(f"dim is {dimension}")
        
#         this_dim = {
#             'sig_ratio': sig_ratio,
#             'dimension': dimension,
#         }
        
#         this_results = []
        
#         passed = False
        
#         Xtrain, Xval = X_train[:,:dimension], X_val[:,:dimension]
#         Ytrain, Yval = y_train,y_val
        
#         train_dataset = simple_dataset(Xtrain, Ytrain)
        
        
        
#         for trial in tqdm(range(4)):
            
#             train_loader = DataLoader(train_dataset, batch_size=20)
            
#             model = MyNet_XL(dimension)
#             model.to(device)

#             optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
            
#             train_acc = 0
#             val_acc = 0
#             success = 0
#             for epoch in (range(MAX_EPOCHS)):
                
#                 train_epoch(model, train_loader)
                
#                 val_acc = validate_acc(model, Xval, Yval)
                                
#                 if (val_acc >= 0.6):
#                     success = 1
#                     this_results.append([dimension,epoch])
#                     break
#             if success:
#                 passed = True
                
#             else:
#                 this_results.append([dimension,-1])
                
#         print(this_results)
#         this_dim['result'] = this_results
        
#         with open(f'data3/model_XL_sig_{sig_ratio}_dim_{dimension}.pickle', 'wb') as handle:
#             pickle.dump(this_dim, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
#         if passed == False:
#             break
            
#         del model

In [64]:
# signalz = [0.005, 0.01,0.015, 0.02, 0.025, 0.03, 0.035, 0.04, 0.045, 0.05, 0.055, 0.06, 0.065, 0.07, 0.075, 0.08, 0.085, 0.09, 0.1]
# dim_lst = [1, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 25, 30, 35, 40, 50, 60, 80, 100]
# data_dir = 'data5'

In [66]:
# for sig_ratio in signalz:
    
#     X_train,y_train,X_val,y_val = physics_dataset.generate_dataset(sig_noise_ratio=sig_ratio)

    
#     print(f"sig ratio is {sig_ratio}")
    
#     for dimension in dim_lst:
#         print(f"dim is {dimension}")
        
#         this_dim = {
#             'sig_ratio': sig_ratio,
#             'dimension': dimension,
#         }
        
#         this_results = []
        
#         passed = False
        
#         Xtrain, Xval = X_train[:,:dimension], X_val[:,:dimension]
#         Ytrain, Yval = y_train,y_val
        
#         train_dataset = simple_dataset(Xtrain, Ytrain)
        
        
        
#         for trial in tqdm(range(10)):
            
#             train_loader = DataLoader(train_dataset, batch_size=20)
            
#             model = MyNet_S(dimension)
#             model.to(device)

#             optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
            
#             train_acc = 0
#             val_acc = 0
#             success = 0
#             for epoch in (range(MAX_EPOCHS)):
                
#                 train_epoch(model, train_loader)
                
#                 val_acc = validate_acc(model, Xval, Yval)
                                
#                 if (val_acc >= 0.6):
#                     success = 1
#                     this_results.append([dimension,epoch])
#                     break
#             if success:
#                 passed = True
                
#             else:
#                 this_results.append([dimension,-1])
                
#         print(this_results)
#         this_dim['result'] = this_results
        
#         with open(f'{data_dir}/model_S_sig_{sig_ratio}_dim_{dimension}.pickle', 'wb') as handle:
#             pickle.dump(this_dim, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
#         if passed == False:
#             break
            
#         del model


# for sig_ratio in signalz:
    
#     X_train,y_train,X_val,y_val = physics_dataset.generate_dataset(sig_noise_ratio=sig_ratio)

    
#     print(f"sig ratio is {sig_ratio}")
    
#     for dimension in dim_lst:
#         print(f"dim is {dimension}")
        
#         this_dim = {
#             'sig_ratio': sig_ratio,
#             'dimension': dimension,
#         }
        
#         this_results = []
        
#         passed = False
        
#         Xtrain, Xval = X_train[:,:dimension], X_val[:,:dimension]
#         Ytrain, Yval = y_train,y_val
        
#         train_dataset = simple_dataset(Xtrain, Ytrain)
        
        
        
#         for trial in tqdm(range(10)):
            
#             train_loader = DataLoader(train_dataset, batch_size=20)
            
#             model = MyNet_M(dimension)
#             model.to(device)

#             optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
            
#             train_acc = 0
#             val_acc = 0
#             success = 0
#             for epoch in (range(MAX_EPOCHS)):
                
#                 train_epoch(model, train_loader)
                
#                 val_acc = validate_acc(model, Xval, Yval)
                                
#                 if (val_acc >= 0.6):
#                     success = 1
#                     this_results.append([dimension,epoch])
#                     break
#             if success:
#                 passed = True
                
#             else:
#                 this_results.append([dimension,-1])
                
#         print(this_results)
#         this_dim['result'] = this_results
        
#         with open(f'{data_dir}/model_M_sig_{sig_ratio}_dim_{dimension}.pickle', 'wb') as handle:
#             pickle.dump(this_dim, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
#         if passed == False:
#             break
            
#         del model

# for sig_ratio in signalz:
    
#     X_train,y_train,X_val,y_val = physics_dataset.generate_dataset(sig_noise_ratio=sig_ratio)

    
#     print(f"sig ratio is {sig_ratio}")
    
#     for dimension in dim_lst:
#         print(f"dim is {dimension}")
        
#         this_dim = {
#             'sig_ratio': sig_ratio,
#             'dimension': dimension,
#         }
        
#         this_results = []
        
#         passed = False
        
#         Xtrain, Xval = X_train[:,:dimension], X_val[:,:dimension]
#         Ytrain, Yval = y_train,y_val
        
#         train_dataset = simple_dataset(Xtrain, Ytrain)
        
        
        
#         for trial in tqdm(range(10)):
            
#             train_loader = DataLoader(train_dataset, batch_size=20)
            
#             model = MyNet_L(dimension)
#             model.to(device)

#             optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
            
#             train_acc = 0
#             val_acc = 0
#             success = 0
#             for epoch in (range(MAX_EPOCHS)):
                
#                 train_epoch(model, train_loader)
                
#                 val_acc = validate_acc(model, Xval, Yval)
                                
#                 if (val_acc >= 0.6):
#                     success = 1
#                     this_results.append([dimension,epoch])
#                     break
#             if success:
#                 passed = True
                
#             else:
#                 this_results.append([dimension,-1])
                
#         print(this_results)
#         this_dim['result'] = this_results
        
#         with open(f'{data_dir}/model_L_sig_{sig_ratio}_dim_{dimension}.pickle', 'wb') as handle:
#             pickle.dump(this_dim, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
#         if passed == False:
#             break
            
#         del model

# for sig_ratio in signalz:
    
#     X_train,y_train,X_val,y_val = physics_dataset.generate_dataset(sig_noise_ratio=sig_ratio)

    
#     print(f"sig ratio is {sig_ratio}")
    
#     for dimension in dim_lst:
#         print(f"dim is {dimension}")
        
#         this_dim = {
#             'sig_ratio': sig_ratio,
#             'dimension': dimension,
#         }
        
#         this_results = []
        
#         passed = False
        
#         Xtrain, Xval = X_train[:,:dimension], X_val[:,:dimension]
#         Ytrain, Yval = y_train,y_val
        
#         train_dataset = simple_dataset(Xtrain, Ytrain)
        
        
        
#         for trial in tqdm(range(10)):
            
#             train_loader = DataLoader(train_dataset, batch_size=20)
            
#             model = MyNet_XL(dimension)
#             model.to(device)

#             optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
            
#             train_acc = 0
#             val_acc = 0
#             success = 0
#             for epoch in (range(MAX_EPOCHS)):
                
#                 train_epoch(model, train_loader)
                
#                 val_acc = validate_acc(model, Xval, Yval)
                                
#                 if (val_acc >= 0.6):
#                     success = 1
#                     this_results.append([dimension,epoch])
#                     break
#             if success:
#                 passed = True
                
#             else:
#                 this_results.append([dimension,-1])
                
#         print(this_results)
#         this_dim['result'] = this_results
        
#         with open(f'{data_dir}/model_XL_sig_{sig_ratio}_dim_{dimension}.pickle', 'wb') as handle:
#             pickle.dump(this_dim, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
#         if passed == False:
#             break
            
#         del model

X_train,y_train,X_val,y_val
sig ratio is 0.005
dim is 1


100%|██████████| 10/10 [06:40<00:00, 40.00s/it]


[[1, -1], [1, -1], [1, -1], [1, -1], [1, -1], [1, -1], [1, 0], [1, -1], [1, -1], [1, -1]]
dim is 2


100%|██████████| 10/10 [07:19<00:00, 44.00s/it]


[[2, -1], [2, -1], [2, -1], [2, -1], [2, -1], [2, -1], [2, -1], [2, -1], [2, -1], [2, -1]]
X_train,y_train,X_val,y_val
sig ratio is 0.01
dim is 1


...

[[6, -1], [6, -1], [6, -1], [6, -1], [6, -1], [6, -1], [6, 11], [6, -1], [6, 15], [6, -1]]
dim is 8


100%|██████████| 10/10 [08:00<00:00, 48.09s/it]


[[8, -1], [8, -1], [8, -1], [8, -1], [8, 12], [8, 15], [8, -1], [8, -1], [8, -1], [8, -1]]
dim is 10


100%|██████████| 10/10 [08:27<00:00, 50.72s/it]

[[10, -1], [10, -1], [10, -1], [10, -1], [10, -1], [10, -1], [10, -1], [10, -1], [10, -1], [10, -1]]


In [16]:
signalz = [0.005, 0.01,0.015, 0.02, 0.025, 0.03, 0.035, 0.04, 0.045, 0.05, 0.055, 0.06, 0.065, 0.07, 0.075, 0.08, 0.085, 0.09, 0.1]
dim_lst = [1, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 25, 30, 35, 40, 50, 60, 80, 100]
data_dir = 'data8'

In [17]:
for sig_ratio in signalz:
    
    X_train,y_train,X_val,y_val = physics_dataset.generate_dataset(sig_noise_ratio=sig_ratio)

    
    print(f"sig ratio is {sig_ratio}")
    
    for big_trial in range(25):
    
        for dimension in dim_lst:
            print(f"dim is {dimension}")

            this_dim = {
                'sig_ratio': sig_ratio,
                'dimension': dimension,
            }

            this_results = []

            passed = False

            Xtrain, Xval = X_train[:,:dimension], X_val[:,:dimension]
            Ytrain, Yval = y_train,y_val

            train_dataset = simple_dataset(Xtrain, Ytrain)




            train_loader = DataLoader(train_dataset, batch_size=20)

            model = MyNet_S(dimension)
            model.to(device)

            optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

            train_acc = 0
            val_acc = 0
            success = 0
            for epoch in (range(MAX_EPOCHS)):

                train_epoch(model, train_loader)

                val_acc = validate_acc(model, Xval, Yval)

                if (val_acc >= 0.6):
                    success = 1
                    this_results.append([dimension,epoch])
                    break
            if success:
                passed = True

            else:
                this_results.append([dimension,-1])

            print(this_results)
            this_dim['result'] = this_results

            with open(f'{data_dir}/model_S_sig_{sig_ratio}_dim_{dimension}_{big_trial}.pickle', 'wb') as handle:
                pickle.dump(this_dim, handle, protocol=pickle.HIGHEST_PROTOCOL)

            if passed == False:
                break

            del model


for sig_ratio in signalz:
    
    X_train,y_train,X_val,y_val = physics_dataset.generate_dataset(sig_noise_ratio=sig_ratio)

    
    print(f"sig ratio is {sig_ratio}")
    for big_trial in range(25):

        for dimension in dim_lst:
            print(f"dim is {dimension}")

            this_dim = {
                'sig_ratio': sig_ratio,
                'dimension': dimension,
            }

            this_results = []

            passed = False

            Xtrain, Xval = X_train[:,:dimension], X_val[:,:dimension]
            Ytrain, Yval = y_train,y_val

            train_dataset = simple_dataset(Xtrain, Ytrain)




            train_loader = DataLoader(train_dataset, batch_size=20)

            model = MyNet_M(dimension)
            model.to(device)

            optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

            train_acc = 0
            val_acc = 0
            success = 0
            for epoch in (range(MAX_EPOCHS)):

                train_epoch(model, train_loader)

                val_acc = validate_acc(model, Xval, Yval)

                if (val_acc >= 0.6):
                    success = 1
                    this_results.append([dimension,epoch])
                    break
            if success:
                passed = True

            else:
                this_results.append([dimension,-1])

            print(this_results)
            this_dim['result'] = this_results

            with open(f'{data_dir}/model_M_sig_{sig_ratio}_dim_{dimension}_{big_trial}.pickle', 'wb') as handle:
                pickle.dump(this_dim, handle, protocol=pickle.HIGHEST_PROTOCOL)

            if passed == False:
                break

            del model

for sig_ratio in signalz:
    
    X_train,y_train,X_val,y_val = physics_dataset.generate_dataset(sig_noise_ratio=sig_ratio)

    
    print(f"sig ratio is {sig_ratio}")
    for big_trial in range(25):
    
        for dimension in dim_lst:
            print(f"dim is {dimension}")

            this_dim = {
                'sig_ratio': sig_ratio,
                'dimension': dimension,
            }

            this_results = []

            passed = False

            Xtrain, Xval = X_train[:,:dimension], X_val[:,:dimension]
            Ytrain, Yval = y_train,y_val

            train_dataset = simple_dataset(Xtrain, Ytrain)




            train_loader = DataLoader(train_dataset, batch_size=20)

            model = MyNet_L(dimension)
            model.to(device)

            optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

            train_acc = 0
            val_acc = 0
            success = 0
            for epoch in (range(MAX_EPOCHS)):

                train_epoch(model, train_loader)

                val_acc = validate_acc(model, Xval, Yval)

                if (val_acc >= 0.6):
                    success = 1
                    this_results.append([dimension,epoch])
                    break
            if success:
                passed = True

            else:
                this_results.append([dimension,-1])

            print(this_results)
            this_dim['result'] = this_results

            with open(f'{data_dir}/model_L_sig_{sig_ratio}_dim_{dimension}_{big_trial}.pickle', 'wb') as handle:
                pickle.dump(this_dim, handle, protocol=pickle.HIGHEST_PROTOCOL)

            if passed == False:
                break

            del model

for sig_ratio in signalz:
    
    X_train,y_train,X_val,y_val = physics_dataset.generate_dataset(sig_noise_ratio=sig_ratio)

    
    print(f"sig ratio is {sig_ratio}")
    for big_trial in range(25):
    
        for dimension in dim_lst:
            print(f"dim is {dimension}")

            this_dim = {
                'sig_ratio': sig_ratio,
                'dimension': dimension,
            }

            this_results = []

            passed = False

            Xtrain, Xval = X_train[:,:dimension], X_val[:,:dimension]
            Ytrain, Yval = y_train,y_val

            train_dataset = simple_dataset(Xtrain, Ytrain)




            train_loader = DataLoader(train_dataset, batch_size=20)

            model = MyNet_XS(dimension)
            model.to(device)

            optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

            train_acc = 0
            val_acc = 0
            success = 0
            for epoch in (range(MAX_EPOCHS)):

                train_epoch(model, train_loader)

                val_acc = validate_acc(model, Xval, Yval)

                if (val_acc >= 0.6):
                    success = 1
                    this_results.append([dimension,epoch])
                    break
            if success:
                passed = True

            else:
                this_results.append([dimension,-1])

            print(this_results)
            this_dim['result'] = this_results

            with open(f'{data_dir}/model_XS_sig_{sig_ratio}_dim_{dimension}_{big_trial}.pickle', 'wb') as handle:
                pickle.dump(this_dim, handle, protocol=pickle.HIGHEST_PROTOCOL)

            if passed == False:
                break

            del model

X_train,y_train,X_val,y_val
sig ratio is 0.005
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
X_train,y_train,X_val,y_val
sig ratio is 0.01
dim is 1
[[1, -1]]
dim is 1
[[1, 1]]
dim is 2
[[2, -1]]
dim is 1
[[1, 2]]
dim is 2
[[2, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, 2]]
dim is 2
[[2, 3]]
dim is 4
[[4, 1]]
dim is 6
[[6, -1]]
dim is 1
[[1, 0]]
dim is 2
[[2, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, 1]]
dim is 2
[[2, 2]]
dim is 4
[[4, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, 2]]
dim is 2
[[2, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim 

In [21]:
# for sig_ratio in signalz:
    
#     X_train,y_train,X_val,y_val = physics_dataset.generate_dataset(sig_noise_ratio=sig_ratio)

    
#     print(f"sig ratio is {sig_ratio}")
#     for big_trial in range(25):
    
#         for dimension in dim_lst:
#             print(f"dim is {dimension}")

#             this_dim = {
#                 'sig_ratio': sig_ratio,
#                 'dimension': dimension,
#             }

#             this_results = []

#             passed = False

#             Xtrain, Xval = X_train[:,:dimension], X_val[:,:dimension]
#             Ytrain, Yval = y_train,y_val

#             train_dataset = simple_dataset(Xtrain, Ytrain)




#             train_loader = DataLoader(train_dataset, batch_size=20)

#             model = MyNet_XS(dimension)
#             model.to(device)

#             optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

#             train_acc = 0
#             val_acc = 0
#             success = 0
#             for epoch in (range(MAX_EPOCHS)):

#                 train_epoch(model, train_loader)

#                 val_acc = validate_acc(model, Xval, Yval)

#                 if (val_acc >= 0.6):
#                     success = 1
#                     this_results.append([dimension,epoch])
#                     break
#             if success:
#                 passed = True

#             else:
#                 this_results.append([dimension,-1])

#             print(this_results)
#             this_dim['result'] = this_results

#             with open(f'{data_dir}/model_XS_sig_{sig_ratio}_dim_{dimension}_{big_trial}.pickle', 'wb') as handle:
#                 pickle.dump(this_dim, handle, protocol=pickle.HIGHEST_PROTOCOL)

#             if passed == False:
#                 break

#             del model

X_train,y_train,X_val,y_val
sig ratio is 0.005
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, 2]]
dim is 2
[[2, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, 0]]
dim is 2
[[2, 6]]
dim is 4
[[4, -1]]
dim is 1
[[1, 2]]
dim is 2
[[2, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, 0]]
dim is 2
[[2, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, 1]]
dim is 2
[[2, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, 3]]
dim is 2
[[2, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, 1]]
dim is 2
[[2, -1]]
dim is 1
[[1, 2]]
dim is 2
[[2, 0]]
dim is 4
[[4, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
X_train,y_train,X_val,y_val
sig ratio is 0.01
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, 2]]
dim is 2
[[2, -1]]
dim is 1
[[1, 1]]
dim is 2
[[2, -1]]
dim is 1
[[1, 3]]
dim is 2
[[2, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, -1]]
dim is 1
[[1, 1]]
dim is 2
[[2, -1]]
dim is 1
[[1, -1]]
dim is 1
